In [1]:

import sys
import operator
import numpy as np
from sklearn import svm
from past import autotranslate
autotranslate(['litcm'])
from litcm import LIT
import re
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf

lit = LIT(labels=['hin', 'eng'], transliteration=False)
#   Dictionary and associated files
en_dict_flname = "en_only"      # English words only
hi_dict_flname = "hi_only"      # Hindi words only
en_2ch_w_flname = "en_2.uniq"   # Most frequent 2-char seq in EN
en_3ch_w_flname = "en_3.uniq"   # Most frequent 3-char seq in EN
hi_2ch_w_flname = "hi_2.uniq"   # Most frequent 2-char seq in HI-roman
hi_3ch_w_flname = "hi_3.uniq"   # Most frequent 3-char seq in HI-roman


# Initialize dictionaries and lists to empty
en_dict = set ()
hi_dict = set ()
en_2ch_w_dict = {}
en_2ch_lst = []
en_3ch_w_dict = {}
en_3ch_lst = []
hi_2ch_w_dict = {}
hi_2ch_lst = []
hi_3ch_w_dict = {}
hi_3ch_lst = []
def load_dict ():
    with open (en_dict_flname) as fl:
        for l in fl:
            en_dict.add (l.strip ())
    
    with open (hi_dict_flname) as fl:
        for l in fl:
            hi_dict.add (l.strip ())

    with open (en_2ch_w_flname) as fl:
        for l in fl:
            arr = [ w for w in l.strip ().split (' ') ]
            en_2ch_w_dict[arr[0]] = float (arr[1])
            en_2ch_lst.append (arr[0])
    
    with open (en_3ch_w_flname) as fl:
        for l in fl:
            arr = [ w for w in l.strip ().split (' ') ]
            en_3ch_w_dict[arr[0]] = float (arr[1])
            en_3ch_lst.append (arr[0])
    
    with open (hi_2ch_w_flname) as fl:
        for l in fl:
            arr = [ w for w in l.strip ().split (' ') ]
            hi_2ch_w_dict[arr[0]] = float (arr[1])
            hi_2ch_lst.append (arr[0])
    
    with open (hi_3ch_w_flname) as fl:
        for l in fl:
            arr = [ w for w in l.strip ().split (' ') ]
            hi_3ch_w_dict[arr[0]] = float (arr[1])
            hi_3ch_lst.append (arr[0])
def load_dict ():
    with open (en_dict_flname) as fl:
        for l in fl:
            en_dict.add (l.strip ())
    
    with open (hi_dict_flname) as fl:
        for l in fl:
            hi_dict.add (l.strip ())

    with open (en_2ch_w_flname) as fl:
        for l in fl:
            arr = [ w for w in l.strip ().split (' ') ]
            en_2ch_w_dict[arr[0]] = float (arr[1])
            en_2ch_lst.append (arr[0])
    
    with open (en_3ch_w_flname) as fl:
        for l in fl:
            arr = [ w for w in l.strip ().split (' ') ]
            en_3ch_w_dict[arr[0]] = float (arr[1])
            en_3ch_lst.append (arr[0])
    
    with open (hi_2ch_w_flname) as fl:
        for l in fl:
            arr = [ w for w in l.strip ().split (' ') ]
            hi_2ch_w_dict[arr[0]] = float (arr[1])
            hi_2ch_lst.append (arr[0])
    
    with open (hi_3ch_w_flname) as fl:
        for l in fl:
            arr = [ w for w in l.strip ().split (' ') ]
            hi_3ch_w_dict[arr[0]] = float (arr[1])
            hi_3ch_lst.append (arr[0])
# Given a sentence 's' return the corresponding feature vector
def mk_vect (s):
    s = s.strip ()
    s_tagged = lit.identify (s)
    ct_en = len (re.findall ('\\Eng', s_tagged))
    ct_hi = len (re.findall ('\\Hin', s_tagged))
    ct_all = ct_en + ct_hi
    
    if ct_all > 0:
        res = [ ct_en/ct_all, ct_hi/ct_all ]
    else:
        res = [ 0.00, 0.00 ]
    v_en = [ 1 if w in en_dict else 0 for w in s.split (' ') ]
    v_hi = [ 1 if w in hi_dict else 0 for w in s.split (' ') ]

    # print (v_en)
    # print (v_hi)
    res.append (sum (v_en) / len (v_en))
    res.append (sum (v_hi) / len (v_hi))

    arr = [ s[i+0] + s[i+1] for i in range(len(s) - 1) ]
    dct = {}
    ct = 0
    for w in arr:
        if w in dct:
            dct[w] += 1
        else:
            dct[w] = 1
        ct += 1

    for k in en_2ch_lst:
        if k in dct:
            res.append (dct[k] * en_2ch_w_dict[k] / ct)
        else:
            res.append (0.00)
    
    arr = [ s[i+0] + s[i+1] for i in range(len(s) - 1) ]
    dct = {}
    ct = 0
    for w in arr:
        if w in dct:
            dct[w] += 1
        else:
            dct[w] = 1
        ct += 1

    for k in hi_2ch_lst:
        if k in dct:
            res.append (dct[k] * hi_2ch_w_dict[k] / ct)
        else:
            res.append (0.00)
    
    arr = [ s[i+0] + s[i+1] + s[i+2] for i in range(len(s) - 2) ]
    dct = {}
    ct = 0
    for w in arr:
        if w in dct:
            dct[w] += 1
        else:
            dct[w] = 1
        ct += 1

    for k in en_3ch_lst:
        if k in dct:
            res.append (dct[k] * en_3ch_w_dict[k] / ct)
        else:
            res.append (0.00)
    
    arr = [ s[i+0] + s[i+1] + s[i+2] for i in range(len(s) - 2) ]
    dct = {}
    ct = 0
    for w in arr:
        if w in dct:
            dct[w] += 1
        else:
            dct[w] = 1
        ct += 1

    for k in hi_3ch_lst:
        if k in dct:
            res.append (dct[k] * hi_3ch_w_dict[k] / ct)
        else:
            res.append (0.00)
    
    return res

def load_data (x_flname, y_flname):
    x_orig = []
    y_orig = []
    with open (x_flname) as fl:
        for l in fl:
            x_orig.append (l.strip ())

    with open (y_flname) as fl:
        for l in fl:
            y_orig.append (int (l.strip ()))

    X = []
    for x in x_orig:
        X.append (mk_vect (x))

    X  = np.array (X)
    Y  = np.array (y_orig)

    return (X, Y, x_orig)

print ("[ ] Loading dictionaries ...", end="")
load_dict ()
print ("\r%-79s" % "[x] Dictionaries loaded.")

[x] Dictionaries loaded.                                                       
